# Decision Tree Regressor

I produce a basic Decision Tree Regressor function and test it on subset of my cleaned UW_Madison GPA dataframe.

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df = pd.read_pickle('for_next_project.pkl').sample(500)
df.head()

,a_perc,ab_perc,b_perc,bc_perc,c_perc,d_perc,f_perc,section_type,start_time,class_length,term_code,class_level,class_meetings,med_enrollment,number_sections
19124,0.500000,0.050000,0.300000,0.050000,0.050000,0.000000,0.050000,LEC,870.0,75.0,1122,3,2.0,18.0,6.0
8538,0.432796,0.233871,0.231183,0.043011,0.021505,0.026882,0.005376,LEC,595.0,50.0,1114,2,3.0,261.0,1.0
61171,0.089552,0.238806,0.402985,0.223881,0.014925,0.000000,0.029851,LEC,595.0,50.0,1102,2,3.0,62.0,2.0
16277,0.227273,0.045455,0.136364,0.090909,0.272727,0.136364,0.090909,LEC,800.0,50.0,1122,3,3.0,17.0,8.0
21130,0.148936,0.148936,0.276596,0.170213,0.180851,0.031915,0.021277,LEC,660.0,50.0,1132,3,3.0,127.0,1.0


### Train test splitting
To begin with, I want to divide the data that I have into training and testing sets. Since the objective here is to build a decision tree that will continue to grow as long as it can continue to reduce variance (subject to possible restrictions on leaf splitting and maximum depth), I'm not going to worry about producing a validation set.

In [3]:
mask_list = np.random.random(len(df))

train_mask = mask_list <= 0.80
test_mask = mask_list > 0.80

train_df = df[train_mask]
test_df = df[test_mask]

print('There are ', len(train_df), ' elements in your training set. \n There are ', len(test_df), ' elements in your test set')

There are  390  elements in your training set. 
 There are  110  elements in your test set


### One hot encoding

Next, I need to one hot encode any categorical variables. We see that in this case, the only true categorical variable that I have is ```section_type```. Below, I've writting one function, ```oh_encoder_fit``` to find the encodings and another ```oh_encoder_transform``` to transform the dataframes. Since a few of the section types occur fairly infrequently, I'm going to cheat a bit and use ```oh_encoder_fit``` on my original dataframe and then transform my test and training dataframes separately.

In [4]:
def oh_encoder_fit(df, cat_vars):
    
#   df1 = df.copy()
    cat_encodes = {}
    
    for cat_var in cat_vars:
        cats = list(set(df[cat_var].values.tolist()))
        cat_encodes[cat_var] = cats
#        for cat in cats:
#            df1[cat] = (df1[cat_var] == cat) * 1
            
#    df1.drop(cat_vars, axis = 1, inplace = True)
                
    return cat_encodes

In [5]:
def oh_encoder_transform(df, cat_encodes):
    
    df1 = df.copy()
    
    for key in cat_encodes.keys():
        cats = cat_encodes[key]
        for cat in cats:
            df1[cat] = (df1[key] == cat) * 1
            
        df1.drop(key, axis = 1, inplace = True)
    
    return df1

In [6]:
# Producing the categorical encodings for the entire data frame.

cat_encodings = oh_encoder_fit(df, ['section_type'])

In [7]:
train_df = oh_encoder_transform(train_df, cat_encodings)
test_df = oh_encoder_transform(test_df, cat_encodings)

In [8]:
train_df.head()

,a_perc,ab_perc,b_perc,bc_perc,c_perc,d_perc,f_perc,start_time,class_length,term_code,class_level,class_meetings,med_enrollment,number_sections,FLD,LEC,DIS,LAB,SEM
19124,0.500000,0.050000,0.300000,0.050000,0.050000,0.000000,0.050000,870.0,75.0,1122,3,2.0,18.0,6.0,0,1,0,0,0
16277,0.227273,0.045455,0.136364,0.090909,0.272727,0.136364,0.090909,800.0,50.0,1122,3,3.0,17.0,8.0,0,1,0,0,0
21130,0.148936,0.148936,0.276596,0.170213,0.180851,0.031915,0.021277,660.0,50.0,1132,3,3.0,127.0,1.0,0,1,0,0,0
4427,0.562500,0.125000,0.125000,0.000000,0.125000,0.062500,0.000000,530.0,50.0,1072,1,4.0,15.0,2.0,0,1,0,0,0
12223,0.210526,0.210526,0.578947,0.000000,0.000000,0.000000,0.000000,570.0,75.0,1134,2,2.0,19.0,21.0,0,1,0,0,0


### Reducing the features used to predict

Looking at the training dataframe above, we see that we actually have 7 response columns and 12 feature columns. To begin with, let's eliminate all but the first response column because I haven't writting a decision tree that can handle multiple features at the same time. Secondly, I want to eliminate ```term_code```, ```start_time```, ```class_level```, and ```class_meetings``` from the feature columns because past experimentation has demonstrated that it actually isn't all that helpful.

In [9]:
train_df.drop(['ab_perc', 'b_perc', 'bc_perc', 'c_perc', 'd_perc', 'f_perc', 'term_code', 'start_time', 'class_level', 'class_meetings'], axis = 1, inplace = True)
test_df.drop(['ab_perc', 'b_perc', 'bc_perc', 'c_perc', 'd_perc', 'f_perc', 'term_code', 'start_time', 'class_level',  'class_meetings'], axis = 1, inplace = True)

In [10]:
train_df.head()

,a_perc,class_length,med_enrollment,number_sections,FLD,LEC,DIS,LAB,SEM
19124,0.500000,75.0,18.0,6.0,0,1,0,0,0
16277,0.227273,50.0,17.0,8.0,0,1,0,0,0
21130,0.148936,50.0,127.0,1.0,0,1,0,0,0
4427,0.562500,50.0,15.0,2.0,0,1,0,0,0
12223,0.210526,75.0,19.0,21.0,0,1,0,0,0


In [11]:
train_data = np.array(train_df)
test_data = np.array(test_df)

In [12]:
train_response = train_data[:, 0]
train_features = train_data[:, 1:]

test_response = test_data[:, 0]
test_features = train_data[:, 1:]

Definition of the impurity reduction for a regression tree.

$$I_{V}(N)={\frac {1}{|S|^{2}}}\sum _{i\in S}\sum _{j\in S}{\frac {1}{2}}(x_{i}-x_{j})^{2}-\left({\frac {1}{|S_{t}|^{2}}}\sum _{i\in S_{t}}\sum _{j\in S_{t}}{\frac {1}{2}}(x_{i}-x_{j})^{2}+{\frac {1}{|S_{f}|^{2}}}\sum _{i\in S_{f}}\sum _{j\in S_{f}}{\frac {1}{2}}(x_{i}-x_{j})^{2}\right)$$

In [13]:
def variance_reduction(response1, response2):
    
    """
    Computes the impurity reduction for a given split in a decision tree.


    Parameters
    ----------
    response1 : list 
        Contains the response values for the first part of the split
    response2 : list
        Contains the response values for the second part of the split

    Returns
    -------
    float
        The reduction (if positive) or increase (if negative) in the impurity with the specified split

    """
    
    data_array = np.concatenate((response1, response2), axis = 0)
    
    sum_da = 0
    for i in range(len(data_array)):
        for j in range(len(data_array)):
            sum_da += 0.5 * (data_array[i] - data_array[j])**2
    #print(sum_da, len(data_array))        
    sum_da = sum_da / (len(data_array))**2
   # print("The sum for the whole array is", sum_da)
    sum_da1 = 0
    for i in range(len(response1)):
        for j in range(len(response1)):
            sum_da1 += 0.5 * (response1[i] - response1[j])**2
    #print(sum_da1, len(response1)    )    
    sum_da1 = sum_da1 / (len(response1))**2
    #print("The sum for response 1 is", sum_da1)
    sum_da2 = 0
    for i in range(len(response2)):
        for j in range(len(response2)):
            sum_da2 += 0.5 * (response2[i] - response2[j])**2
    #print(sum_da2, len(response2)     )   

    sum_da2 = sum_da2 / (len(response2))**2
    #print("The sum for response 2 is", sum_da2)


    reduction = sum_da - (sum_da1 + sum_da2)
    
    return reduction

In [14]:
def find_best_split_one(da, col, response):
    
    """
    Finds the best split for a single column (feature) in a data array


    Parameters
    ----------
    da : numpy array 
        Contains the feature values
    col : int
        The column of interest
    response : list
        Contains the response values

    Returns
    -------
    float or None
        The best value at which to split for the specified column, 
        None if none of the possible splits produce a reduction in impurity
    float
        The reduction in impurity with the best split for the specified column
        0 if none of the possible splits produce a reduction in impurity
        
    """
    
    # Here we are looking for the best split for a single variable for a regression problem
    
    
    current_red = 0
    
    to_split = None
    values = sorted(list(set(da[:, col])))
    #print("the values found for column {} are".format(col), values)

    for i in range(len(values) - 1):
        keys_a = [val for val in (da[:, col] <= values[i])]
        resp_a = response[keys_a]
        #print(len(resp_a))
        keys_b = [val for val in (da[:, col] > values[i])]
        resp_b = response[keys_b]
       # print(len(resp_b))
        new_red = variance_reduction(resp_a, resp_b)
        
        if new_red > current_red:
            current_red = new_red
            to_split = values[i]

    return to_split, current_red

In [15]:
def find_best_split(da, response):
    
    """
    Finds the best split across all possible variables in a data array


    Parameters
    ----------
    da : numpy array 
        Contains the feature values
    response : list
        Contains the response values

    Returns
    -------
    int
        The best column on which to split
    float or None
        The best value at which to split for the best column, 
        None if there are no possible splits which produce a reduction in impurity
    float
        The reduction in impurity with the best split for the specified column
        0 if there are no possible splits which produce a reduction in impurity
    
    """
    
    num_cols = da.shape[1]
    best_current_red = 0
    best_split = None
    var_to_split = None
    var_type = None
    for col in range(num_cols):
        try:
            to_split, new_red = find_best_split_one(da, col, response)
        except:
            to_split = None
            new_red = 0
        if new_red > best_current_red:
            best_current_red = new_red
            var_to_split = col
            best_split = to_split
    
    return var_to_split, best_split, best_current_red

In [16]:
def combine_masks(mask1, mask2):
    
    """
    Combines two boolean masks in an 'AND' fashion, so that the resulting mask
    is True exactly where the two input masks were true


    Parameters
    ----------
    mask1 : list 
        Contains boolean values
    mask2 : list
        Contains boolean values

    Returns
    -------
    list
        A boolean combination of the input masks
    
    """

    new_mask = mask1*mask2
    new_mask = [True if value == 1 else False for value in new_mask]
    
    return new_mask

In [17]:
def split_data(da, split_list):
    
    """
    Finds a subarray of a data array which satisfies the conditions of the specified list
    of data splits


    Parameters
    ----------
    da : numpy array 
        Contains the feature values
    split_list : list
        A list of splits to follow to create a leaf of a decision tree. Each split is a list
        of the form [column, value, side] where:
        
        column: int
            the column we are splitting based on
        value: float
            the decision value for the column
        side : 0 or 1
            if side = 0, we keep values <= value
            if side = 1, we keep values > value

    Returns
    -------
    numpy array
        Contains the rows of the input data array which satisfied each split in order
    list
        A boolean list indicating which rows of the data array were retained
    
    """


    # The split list is a list of the splits to follow to create a leaf of the current decision tree
    # It contains lists and each of the interior lists is of the form [column, value, side (0, 1)]
    
    da_split = da.copy()
    mask = [True for i in range(da.shape[0])]  
    
    for i in range(len(split_list)):
        col, val, side = split_list[i]
        if side == 0:
            da_split = da_split[da_split[:, col] <= val]
            keepers = da[:, col] <= val
            mask = combine_masks(mask, keepers)
        else:
            da_split = da_split[da_split[:, col] > val]
            keepers = da[:, col] > val
            mask = combine_masks(mask, keepers)
                
    return da_split, mask

In [18]:
def decision_tree(da, response, min_splitting = 2, max_depth = 10):

    """
    Computes the leaves of a decision tree


    Parameters
    ----------
    da : numpy array 
        Contains the feature values
    response: list
        Contains the response values
    min_splitting : int
        The minimum number of elements required to split a leaf, default value is 2
    max_depth : int
        The maximum depth of a leaf, default value is 10

    Returns
    -------
    list
        A list of lists of splits to follow for each leaf. 
        Each split is of the form [column, value, side] where:
        
        column: int
            the column we are splitting based on
        value: float
            the decision value for the column
        side : 0 or 1
            if side = 0, we keep values <= value
            if side = 1, we keep values > value
    """

    splits_so_far = []
    
    keep_going = True
    
    # Finding the first split
    
    var_to_split, best_split, best_current_red = find_best_split(da, response)
    
    if best_current_red > 0:
        splits_so_far = [[[var_to_split, best_split, 0]], 
                     [[var_to_split,best_split, 1]]]
    else:
        keep_going = False

    while keep_going:
        # I'm going to need to consider some number of possible leaves here, where at
        # most these are going to be equal to the product of the number of possible values

        current_index = -1
        current_split = None
        current_cat = None
        current_var = None
        current_decr = 0
        
        # Finding (hopefully) the best split among the possibilities
        for i in range(len(splits_so_far)):
            
            if len(splits_so_far[i]) < max_depth:
                split_da, mask = split_data(da, splits_so_far[i])

                if len(split_da) >= min_splitting:
                    var_to_split, best_split, best_current_red = find_best_split(split_da, response[mask])
                    if best_current_red > current_decr:
                        current_index = i
                        current_split = best_split
                        current_var = var_to_split
        
        # And adding its data to my branching tree...
        
        if current_split != None:
            splits_so_far.insert(current_index, splits_so_far[current_index].copy())
            splits_so_far[current_index].append([current_var, current_split, 0])
            splits_so_far[current_index + 1].append([current_var, current_split, 1])
                    
        else:
            keep_going = False
        
    return splits_so_far

In [19]:
def leaf_mean(da, split, response):
    
    """
    Computes the mean of the responses for a specified leaf


    Parameters
    ----------
    da : list 
        Contains the feature values
        
    split : list
        A list of lists of splits to follow for each leaf. 
        Each split is of the form [column, value, side] where:
        
        column: int
            the column we are splitting based on
        value: float
            the decision value for the column
        side : 0 or 1
            if side = 0, we keep values <= value
            if side = 1, we keep values > value
            
    response : list
        Contains the response values
        
    Returns
    -------
    dictionary
        The means of the responses for the data retained in each leaf
    
    """
    
    means = {}
    
    for i in range(len(split)):
        leaf_da, mask = split_data(da, split[i])
        mean = np.mean(response[mask])
        means[i] = mean
        
    return means

In [20]:
def verify_split(point, split):

    """
    Determines whether or not a given point belongs to a given leaf


    Parameters
    ----------
    point : list 
        Contains the feature values for a single observation
        
    split : list
        Contains the splits to perform. Each split is of the form [column, value, side] where:
        
        column: int
            the column we are splitting based on
        value: float
            the decision value for the column
        side : 0 or 1
            if side = 0, we keep values <= value
            if side = 1, we keep values > value
                    
    Returns
    -------
    boolean
        True if the observation belongs to the specified leaf, 
        False otherwise
    
    """

    col, val, side = split
    if side == 0:
        if point[col] <= val:
            return True
        else:
            return False
    else:
        if point[col] > val:
            return True
        else:
            return False

In [21]:
def predict_one_value(test_da, row, leaf_splits, means):

    """
    Predicts the value for a observation, that is to say for a single row of a test array


    Parameters
    ----------
    test_da : numpy array 
        Contains the feature values the test data
        
    row : int
        The row number of the observation for which we are predicting an outcome
        
    leaf_splits : list
        A list of lists of splits to follow for each leaf. 
        Each split is of the form [column, value, side] where:
        
        column: int
            the column we are splitting based on
        value: float
            the decision value for the column
        side : 0 or 1
            if side = 0, we keep values <= value
            if side = 1, we keep values > value
            
    means : dictionary
        The means of the responses for the data retained in each leaf
                    
    Returns
    -------
    float
        The predicted response for the given observation
    
    """
        
    leaf = None
        
    test_point = test_da[row, :]
    
    for j in range(len(leaf_splits)):

        current_leaf = leaf_splits[j]
    
        tests = []
        for i in range(len(current_leaf)):
            tests.append(verify_split(test_point, current_leaf[i]))
    
        if False not in tests:
            leaf = j
            break
        
    return means[leaf]
    

In [22]:
def predictions(test_da, leaf_splits, means):

    """
    Predicts the value for all observations in an array


    Parameters
    ----------
    test_da : numpy array 
        Contains the feature values the test data
                
    leaf_splits : list
        A list of lists of splits to follow for each leaf. 
        Each split is of the form [column, value, side] where:
        
        column: int
            the column we are splitting based on
        value: float
            the decision value for the column
        side : 0 or 1
            if side = 0, we keep values <= value
            if side = 1, we keep values > value
            
    means : dictionary
        The means of the responses for the data retained in each leaf
                    
    Returns
    -------
    list
        The predicted responses for the given observations
    
    """

    predicted = []
    
    for i in range(test_da.shape[0]):
        prediction = predict_one_value(test_da, i, leaf_splits, means)
        predicted.append([prediction])
            
    return predicted

In [23]:
def MSE(y_true, y_pred):

    """
    Computes the mean squared error of predicted responses


    Parameters
    ----------
    y_true : list 
        The observed values of the response variable
        
    y_pred : list
        The predicted values of the response variable
                            
    Returns
    -------
    float
        The mean squared error of the predicted values
    
    """


    squared_diff = [(y_true[i] - y_pred[i])**2 for i in range(len(y_true))]
    sum_of_squares = sum(squared_diff)
    
    mse = sum_of_squares/len(y_true)
    
    return mse

In [24]:
def MAE(y_true, y_pred):

    """
    Computes the mean absolute error of predicted responses


    Parameters
    ----------
    y_true : list 
        The observed values of the response variable
        
    y_pred : list
        The predicted values of the response variable
                            
    Returns
    -------
    float
        The mean absolute error of the predicted values
    
    """

    abs_diff = [abs(y_true[i] - y_pred)[i] for i in range(len(y_true))]
    sum_of_abs_diff = sum(abs_diff)
    
    mae = sum_of_abs_diff/len(y_true)
    
    return mae

In [25]:
splits = decision_tree(train_features, train_response)

In [26]:
means = leaf_mean(train_features, splits, train_response)

In [27]:
pred_response = predictions(test_features, splits, means)

In [28]:
mean_squared_error = MSE(test_response, pred_response)
mean_absolute_error = MAE(test_response, pred_response)

print('The mean squared error in this prediction is: {}'.format(np.around(mean_squared_error, 3)))
print('The mean absolute error in this prediction is: {}'.format(np.around(mean_absolute_error, 3)))

The mean squared error in this prediction is: [0.075]
The mean absolute error in this prediction is: [0.234]
